# Machine Learning for Seagrass Extent Prediction

In [1]:
from pystac.client import Client
from odc.stac import load
import geopandas as gpd
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import joblib
from matplotlib import colors
from sklearn.ensemble import RandomForestClassifier 
import seaborn as sns
from scipy.stats import norm


from utils_test import scale, do_prediction, calculate_band_indices, apply_masks, threshold_calc_land, threshold_calc_ds

## Loading Sentinel-2 GeoMAD

Load data, then create band indices and mask out areas we don't want to include.

In [2]:
catalog = "https://stac.digitalearthpacific.org"
client = Client.open(catalog)

In [3]:
#version = "funafuti_mask_test4"

In [4]:
gdf = gpd.read_file("training-data/marovo.geojson")
#version = "malekula_joined_model_mask_test3"
#gdf = gpd.read_file("training-data/Alluvium_vu_malekula_13022025.geojson")
gdf = gdf.to_crs("EPSG:4326")
min_lon, min_lat, max_lon, max_lat = gdf.total_bounds
bbox = [min_lon, min_lat, max_lon, max_lat]
datetime="2024"

In [5]:
items = client.search(
    collections=["dep_s2_geomad"], datetime=datetime, bbox=bbox
).item_collection()

print(f"Found {len(items)} items")

Found 1 items


In [6]:
data = load(
    items,
    bbox=bbox,
    measurements=[
        "nir",
        "red",
        "blue",
        "green",
        "emad",
        "smad",
        "bcmad",
        "count",
        "green",
        "nir08",
        "nir09",
        "swir16",
        "swir22",
        "coastal",
        "rededge1",
        "rededge2",
        "rededge3",
    ],
    chunks={"x": 2048, "y": 2048},
)

data

<xarray.Dataset> Size: 12MB
Dimensions:      (y: 438, x: 696, time: 1)
Coordinates:
  * y            (y) float64 4kB -9.421e+05 -9.421e+05 ... -9.465e+05 -9.465e+05
  * x            (x) float64 6kB 8.891e+05 8.891e+05 ... 8.96e+05 8.961e+05
    spatial_ref  int32 4B 3832
  * time         (time) datetime64[ns] 8B 2024-01-01
Data variables: (12/16)
    nir          (time, y, x) uint16 610kB dask.array<chunksize=(1, 438, 696), meta=np.ndarray>
    red          (time, y, x) uint16 610kB dask.array<chunksize=(1, 438, 696), meta=np.ndarray>
    blue         (time, y, x) uint16 610kB dask.array<chunksize=(1, 438, 696), meta=np.ndarray>
    green        (time, y, x) uint16 610kB dask.array<chunksize=(1, 438, 696), meta=np.ndarray>
    emad         (time, y, x) float32 1MB dask.array<chunksize=(1, 438, 696), meta=np.ndarray>
    smad         (time, y, x) float32 1MB dask.array<chunksize=(1, 438, 696), meta=np.ndarray>
    ...           ...
    swir16       (time, y, x) uint16 610kB dask.array<chunksize=(1, 438, 696), meta=np.ndarray>
    swir22       (time, y, x) uint16 610kB dask.array<chunksize=(1, 438, 696), meta=np.ndarray>
    coastal      (time, y, x) uint16 610kB dask.array<chunksize=(1, 438, 696), meta=np.ndarray>
    rededge1     (time, y, x) uint16 610kB dask.array<chunksize=(1, 438, 696), meta=np.ndarray>
    rededge2     (time, y, x) uint16 610kB dask.array<chunksize=(1, 438, 696), meta=np.ndarray>
    rededge3     (time, y, x) uint16 610kB dask.array<chunksize=(1, 438, 696), meta=np.ndarray>

# Before Masking 

In [7]:
scaled_data = scale(data)
indices = calculate_band_indices(scaled_data)

scaled_data.odc.explore(bands=["red", "green", "blue"], vmin=0, vmax=0.3)

/srv/conda/envs/notebook/lib/python3.10/site-packages/rasterio/warp.py:387: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dest = _reproject(
/srv/conda/envs/notebook/lib/python3.10/site-packages/rasterio/warp.py:387: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dest = _reproject(


# Masking

In [8]:

# Normal distribution of band, used this to check for distribution  

ln_bg = scaled_data['ln_bg'].values.flatten()  # flatten in case it's multi-dimensional

# Remove NaNs if present
ln_bg = ln_bg[~np.isnan(ln_bg)]

# Calculate mean and std
mean = ln_bg.mean()
std = ln_bg.std()

# Plot histogram
plt.figure(figsize=(15, 5))
sns.histplot(ln_bg, bins=60, kde=False, stat='count', color='skyblue', label='Histogram')

# Plot fitted normal distribution
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 10)
p = norm.pdf(x, mean, std)
plt.plot(x, p, 'r', linewidth=2, label='Normal fit')

plt.title('Normal Distribution')
plt.xlabel('ln_bg')
plt.ylabel('count')
plt.legend()
plt.show()

In [9]:
#Masking with Threshhold function-------Land Mask

data = scaled_data

#Land Mask 0,-0.35,-0.5

mndwi = data['mndwi']
###function goes here##### #parameters = specific band, index , and level (Low,Mid,High)
level = "Low"
thresh_value = threshold_calc_land(mndwi,level) 
####################
print(thresh_value)
mndwi_land_mask = mndwi > thresh_value 
masked_data = data.where(mndwi_land_mask)

-0.155


In [10]:
#Masking with Threshhold function-------Deep sea Mask

bg = masked_data["ln_bg"]
###function goes here##### #parameters = specific band, index , and level (Low,Mid,High)
level = "Low"
thresh_value = threshold_calc_ds(bg,level) #-0.222
#print(thresh_value)
mask_bg = bg < thresh_value
masked_data  = data.where(mask_bg)



In [11]:
masked_data.odc.explore(bands=["red", "green", "blue"], vmin=0, vmax=0.3)

In [13]:
masked_data = masked_data.drop_vars(['stumpf', 'ln_bg'])
masked_data

<xarray.Dataset> Size: 57MB
Dimensions:      (time: 1, y: 438, x: 696)
Coordinates:
  * y            (y) float64 4kB -9.421e+05 -9.421e+05 ... -9.465e+05 -9.465e+05
  * x            (x) float64 6kB 8.891e+05 8.891e+05 ... 8.96e+05 8.961e+05
    spatial_ref  int32 4B 3832
  * time         (time) datetime64[ns] 8B 2024-01-01
Data variables: (12/25)
    nir          (time, y, x) float64 2MB dask.array<chunksize=(1, 438, 696), meta=np.ndarray>
    red          (time, y, x) float64 2MB dask.array<chunksize=(1, 438, 696), meta=np.ndarray>
    blue         (time, y, x) float64 2MB dask.array<chunksize=(1, 438, 696), meta=np.ndarray>
    green        (time, y, x) float64 2MB dask.array<chunksize=(1, 438, 696), meta=np.ndarray>
    emad         (time, y, x) float32 1MB dask.array<chunksize=(1, 438, 696), meta=np.ndarray>
    smad         (time, y, x) float32 1MB dask.array<chunksize=(1, 438, 696), meta=np.ndarray>
    ...           ...
    ndvi         (time, y, x) float64 2MB dask.array<chunksize=(1, 438, 696), meta=np.ndarray>
    ndwi         (time, y, x) float64 2MB dask.array<chunksize=(1, 438, 696), meta=np.ndarray>
    b_g          (time, y, x) float64 2MB dask.array<chunksize=(1, 438, 696), meta=np.ndarray>
    b_r          (time, y, x) float64 2MB dask.array<chunksize=(1, 438, 696), meta=np.ndarray>
    mci          (time, y, x) float64 2MB dask.array<chunksize=(1, 438, 696), meta=np.ndarray>
    ndci         (time, y, x) float64 2MB dask.array<chunksize=(1, 438, 696), meta=np.ndarray>

# Run predictions over our region

In [14]:
model = joblib.load("models/model-geomad-joined-data-rf-11042025.model")

In [15]:
loaded = masked_data.compute()

In [16]:
loaded

<xarray.Dataset> Size: 16GB
Dimensions:      (time: 1, y: 8727, x: 9449)
Coordinates:
  * y            (y) float64 70kB -1.775e+06 -1.775e+06 ... -1.862e+06
  * x            (x) float64 76kB 1.894e+06 1.894e+06 ... 1.988e+06 1.988e+06
    spatial_ref  int32 4B 3832
  * time         (time) datetime64[ns] 8B 2024-01-01
Data variables: (12/25)
    nir          (time, y, x) float64 660MB nan 0.0229 nan ... 0.0384 0.039
    red          (time, y, x) float64 660MB nan 0.0261 nan ... 0.0418 0.0428
    blue         (time, y, x) float64 660MB nan 0.0501 nan ... 0.0631 0.0632
    green        (time, y, x) float64 660MB nan 0.0327 nan ... 0.0483 0.0481
    emad         (time, y, x) float32 330MB nan 0.06707 nan ... 0.1051 0.1082
    smad         (time, y, x) float32 330MB nan 2.988e-06 ... 1.589e-06 1.4e-06
    ...           ...
    ndvi         (time, y, x) float64 660MB nan -0.06531 ... -0.04239 -0.04645
    ndwi         (time, y, x) float64 660MB nan 0.1763 nan ... 0.1142 0.1045
    b_g          (time, y, x) float64 660MB nan 1.532 nan ... 1.285 1.306 1.314
    b_r          (time, y, x) float64 660MB nan 1.92 nan ... 1.452 1.51 1.477
    mci          (time, y, x) float64 660MB nan 0.8388 nan ... 0.844 0.8405
    ndci         (time, y, x) float64 660MB nan 0.02247 nan ... 0.04238 0.04036

In [ ]:
predicted_da = do_prediction(loaded, model).astype(np.float32)
predicted_da

In [ ]:
# gdf = gpd.read_file("training-data/Alluvium_vu_malekula_13022025.geojson")
# gdf = gdf.to_crs("EPSG:4326")
# min_lon, min_lat, max_lon, max_lat = gdf.total_bounds
# bbox = [min_lon, min_lat, max_lon, max_lat]
# datetime="2024"

In [ ]:
# Define training data
#gdf = gpd.read_file("training-data/Alluvium_vu_malekula_13022025.geojson")
#gdf = gdf.to_crs("EPSG:4326")
# gdf.explore(column="cc_id", legend=True)

In [ ]:
postcard =data #.where(all_masks)
# postcard =clipped_ds.to_array(dim="band")
# First transform the training points to the same CRS as the data
training = gdf.to_crs(postcard.odc.geobox.crs)

In [ ]:
print(training.columns)
#training=training.drop(columns=['date', 'uuid'])

In [ ]:
training

In [ ]:
training_da = training.assign(x=training.geometry.x, y=training.geometry.y).to_xarray()

In [ ]:
training_values = (
    postcard.sel(training_da[["x", "y"]], method="nearest").squeeze().compute().to_pandas()
)
training_values

In [ ]:
# Join the training data with the extracted values and remove unnecessary columns
training_array = pd.concat([training["cc_id"], training_values], axis=1)
# Drop rows where there was no data available
training_array = training_array.dropna()
# Preview our resulting training array
training_array.head()

In [ ]:
postcard_df = training_array
postcard_df.columns

In [ ]:
postcard_df.columns

In [ ]:
# postcard_df.columns
# postcard_df = postcard_df.drop(columns=["spatial_ref", "time", "field_1", "y", "x"])
# postcard_df = postcard_df.drop(columns=["field_1"])
# postcard_df = postcard_df.drop(columns=["y", "x", "spatial_ref"])

postcard_df = postcard_df.drop(columns=["time"])
#postcard_df

In [ ]:
postcard_df

In [ ]:
len(postcard_df.columns)

In [ ]:
# The classes are the first column
classes = np.array(postcard_df)[:, 0]

# The observation data is everything after the second column
observations = np.array(postcard_df)[:, 1:]

# Create a model...
classifier = RandomForestClassifier()

# ...and fit it to the data
model = classifier.fit(observations, classes)

In [ ]:
postcard_params = postcard_df.columns[1:]
print(postcard_params)

In [ ]:


importances = classifier.feature_importances_

# Create the DataFrame
data = {
    'param': postcard_params,  # Use the list directly
    'importance': importances,  # Use the list directly
}
importance_df = pd.DataFrame(data)

importance_df["%"] = (importance_df["importance"]*100)

sorted_df = importance_df.sort_values(by='importance', ascending=False)
sorted_df.to_csv(f"{version}importance_df.csv")

sorted_df
# sorted_df.to_csv(.csv', index=False)

In [ ]:
sorted_df


# Visualisation
Visualise the prediction with the 8 class colour map or the 10 class colour map below, respectively

In [ ]:
classes = [
    [1, "sediment", "#8c8c8c"],
    [2, "sand", "#fedd24"],
    [3, "rubble", "#f8ffb4"],
    [4, "seagrass", "#6df7dc"],
    [5, "seaweed", "#b9df6f"],
    [6, "coral", "#a011c3"],
    [7, "rock", "#804600"],
    [8, "deeps", "#011b61"],
]

values_list = [c[0] for c in classes]
color_list = [c[2] for c in classes]

# Build a listed colormap.
c_map = colors.ListedColormap(color_list)
bounds = values_list + [8]
norm = colors.BoundaryNorm(bounds, c_map.N)
predicted_da.odc.explore(cmap=c_map)

In [ ]:
classes = [
    [1, "sediment", "#8c8c8c"],
    [2, "sand", "#fedd24"],
    [3, "rubble", "#f8ffb4"],
    [4, "seagrass", "#6df7dc"],
    [5, "seaweed", "#b9df6f"],
    [6, "coral", "#a011c3"],
    [7, "rock", "#804600"],
    [8, "deeps", "#011b61"],
    [9, "mangrove", "#086a39"],
    [10, "land", "#00FFFFFF"],
]

values_list = [c[0] for c in classes]
color_list = [c[2] for c in classes]

# Build a listed colormap.
c_map = colors.ListedColormap(color_list)
bounds = values_list + [14]
norm = colors.BoundaryNorm(bounds, c_map.N)

In [ ]:
predicted_da.odc.explore(cmap=c_map)

In [ ]:
version = "Tues"

In [ ]:
# predicted_da.odc.write_cog("predictions/predicted_ba_estuary_joined_data_postcard_svm_22042025.tiff", overwrite=True)
#predicted_da.odc.write_cog("predictions/predicted_komave_postcard_rf_24042025.tiff", overwrite=True)
predicted_da.odc.write_cog(f"predictions/{version}.tiff", overwrite=True)

In [ ]:
#predicted_da.rio.to_raster(f"predictions/{version}.tiff")
predicted_da_band = predicted_da.squeeze(drop=True)
predicted_da_band.rio.to_raster("output2.tiff")

# Quantitative validation (to be completed)

In [ ]:
test_data = gpd.read_file("testing-data/utanglang_postcard.geojson")
test_data.explore()

In [ ]:
print("stop here")

In [ ]:
stop_here


In [ ]:
# First transform the training points to the same CRS as the data
test = test_data.to_crs(postcard.odc.geobox.crs)
# Next get the X and Y values out of the point geometries
training_da = test.assign(x=test.geometry.x, y=test.geometry.y).to_xarray()
# Now we can use the x and y values (lon, lat) to extract values from the median composite
training_values = (
    postcard.sel(training_da[["x", "y"]], method="nearest").squeeze().compute().to_pandas()
)
len(training_values)

# Join the training data with the extracted values and remove unnecessary columns
training_array = pd.concat([test["observed_id"], training_values], axis=1)
training_array = training_array.drop(
    columns=[
        "y",
        "x",
        "spatial_ref",
    ]
)
# # Drop rows where there was no data available
# training_array = training_array.dropna()
# Preview our resulting training array
training_array.head()

In [ ]:
# Join the training data with the extracted values and remove unnecessary columns
training_array = pd.concat([test["observed_id"], training_values], axis=1)
training_array = training_array.drop(
    columns=[
        "y",
        "x",
        "spatial_ref",
    ]
)
# # Drop rows where there was no data available
# training_array = training_array.dropna()
# Preview our resulting training array
training_array.head()

In [ ]:
print(predicted_da.dtype)  # Check the dtype of your DataArray
predicted_da = predicted_da.astype('float32')  # Convert to float32

In [ ]:
print(len(training_array), len(test))  # Check the lengths of both arrays

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

# Sample data
np.random.seed(42)
training_array = np.random.rand(100, 5)
test = pd.DataFrame({"observed_id": np.random.choice([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 100)})

# Split the data
X_train, X_test, y_train, y_test = train_test_split(training_array, test.observed_id, test_size=0.9, random_state=42)

# Replace None values with a default value, e.g., 0 or the most frequent value
y_train = y_train.fillna(0)  # or y_train.fillna(y_train.mode()[0])
y_test = y_test.fillna(0)    # Ensure y_test also has no None values
y_train = y_train.astype(int)
y_test = y_test.astype(int)

# Train your model
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Generate and display confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_display = ConfusionMatrixDisplay(confusion_matrix=cm)
plt.figure(figsize=(12, 10))  # Adjust width and height for larger plot
cm_display.plot(cmap=plt.cm.Blues)
plt.show()

In [ ]:
cm

In [ ]:
from sklearn.metrics import accuracy_score

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

print(f'Accuracy: {accuracy:.2f}')

In [ ]:
from sklearn.metrics import classification_report

# report = classification_report(true_labels, predicted_labels, target_names=class_labels)
# print(report)

report = classification_report(y_test, y_pred, target_names=['sediment', 'sand', 'rubble', 'seagrass', 'seaweed', 'coral', 'rock', 'deeps', 'mangrove', 'land'])
print(report)

In [ ]:
print("Unique labels in y_test:", np.unique(y_test))
print("Unique labels in y_pred:", np.unique(y_pred))

In [ ]:
print("Length of y_test:", len(y_test))
print("Length of y_pred:", len(y_pred))